### Metro Manila Flood Insights – Feature Engineering & Preprocessing
This notebook refines and extends the raw dataset to improve predictive modeling and dashboarding.
All steps include short, intuitive justifications suitable for non-technical stakeholders.


In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

# Load raw data
raw_path = ("C:/Users/SHANIA/Downloads/Metro-Manila-Flood-Insights/Data/AEGISDataset.csv")
df = pd.read_csv(raw_path)
df.head(5)


,lat,lon,flood_heig,elevation,precipitat
0,14.640394,121.055708,0,54.553295,9.0
1,14.698299,121.002132,0,21.856272,10.0
2,14.698858,121.100261,0,69.322807,16.0
3,14.571310,120.983334,0,10.987241,8.0
4,14.762232,121.075735,0,87.889847,18.0


We load the raw dataset into memory. This is the starting point for all analysis. Keeping the raw file untouched ensures we always have a reference baseline.

In [2]:
# Data cleaning
print("Missing values per column:\n", df.isnull().sum())
duplicates_count = df.duplicated().sum()
print(f"Duplicate rows: {duplicates_count}")

if duplicates_count > 0:
    df = df.drop_duplicates().reset_index(drop=True)


Missing values per column:
 lat           0
lon           0
flood_heig    0
elevation     0
precipitat    0
dtype: int64
Duplicate rows: 2


Checking for missing values and duplicates ensures our analysis is based on clean, reliable data.


In [3]:
# Binning elevation and precipitation
df['elevation_bin'] = pd.cut(
    df['elevation'], bins=[-1,5,15,30,50,100],
    labels=['Very Low','Low','Moderate','High','Very High']
)
df['precip_bin'] = pd.cut(
    df['precipitat'], bins=[-1,5,10,15,20,25],
    labels=['Very Low','Low','Moderate','High','Very High']
)

df[['elevation','elevation_bin','precipitat','precip_bin']].head(10)


,elevation,elevation_bin,precipitat,precip_bin
0,54.553295,Very High,9.0,Low
1,21.856272,Moderate,10.0,Low
2,69.322807,Very High,16.0,High
3,10.987241,Low,8.0,Low
4,87.889847,Very High,18.0,High
5,14.463734,Low,11.0,Moderate
6,45.000553,High,12.0,Moderate
7,43.226852,High,13.0,Moderate
8,43.183113,High,12.0,Moderate
9,48.276211,High,17.0,High


Grouping locations into categories makes charts and maps more readable for dashboards.


In [4]:
# Flood Risk Index (heuristic)
df['flood_risk_index'] = (
    df['flood_heig'] +
    (1 - df['elevation']/df['elevation'].max()) +
    df['precipitat']/df['precipitat'].max()
)
df['flood_risk_index_scaled'] = df['flood_risk_index'] / df['flood_risk_index'].max()

df[['flood_heig','elevation','precipitat','flood_risk_index_scaled']].head(10)


,flood_heig,elevation,precipitat,flood_risk_index_scaled
0,0,54.553295,9.0,0.087329
1,0,21.856272,10.0,0.129570
2,0,69.322807,16.0,0.105380
3,0,10.987241,8.0,0.131989
4,0,87.889847,18.0,0.094189
5,0,14.463734,11.0,0.142977
6,0,45.000553,12.0,0.113163
7,0,43.226852,13.0,0.120167
8,0,43.183113,12.0,0.115234
9,0,48.276211,17.0,0.134346


A single score summarizing flood vulnerability allows quick comparison across areas in dashboards.


In [5]:
# Spatial clustering
n_clusters = 20
df['cluster_id'] = KMeans(n_clusters=n_clusters, random_state=42).fit_predict(df[['lat','lon']])

# Compute cluster-level statistics
cluster_stats = df.groupby('cluster_id')[['elevation','precipitat','flood_heig']].mean().reset_index()
cluster_stats.rename(columns={
    'elevation':'cluster_mean_elev',
    'precipitat':'cluster_mean_precip',
    'flood_heig':'cluster_mean_flood'
}, inplace=True)

df = df.merge(cluster_stats, on='cluster_id', how='left')
df[['cluster_id','cluster_mean_elev','cluster_mean_precip','cluster_mean_flood']].head(10)


,cluster_id,cluster_mean_elev,cluster_mean_precip,cluster_mean_flood
0,17,52.868351,10.234310,2.037657
1,14,12.199577,9.056338,1.915493
2,10,31.323989,15.278351,2.948454
3,19,9.217746,7.972222,2.384259
4,4,54.221978,13.095238,2.440476
5,18,7.961419,11.101167,1.677043
6,4,54.221978,13.095238,2.440476
7,4,54.221978,13.095238,2.440476
8,4,54.221978,13.095238,2.440476
9,10,31.323989,15.278351,2.948454


Clusters summarize local neighborhoods so dashboards can highlight the most at-risk areas.


In [6]:
# Domain-inspired interaction features
df['elev_precip_interaction'] = df['precipitat'] / (df['elevation'] + 0.1)
df['low_elev_high_rain'] = ((df['elevation'] < 10) & (df['precipitat'] > 15)).astype(int)
df['very_low_elev'] = (df['elevation'] < 5).astype(int)
df['very_high_rain'] = (df['precipitat'] > 18).astype(int)

df[['elevation','precipitat','elev_precip_interaction','low_elev_high_rain','very_low_elev','very_high_rain']].head(10)


,elevation,precipitat,elev_precip_interaction,low_elev_high_rain,very_low_elev,very_high_rain
0,54.553295,9.0,0.164674,0,0,0
1,21.856272,10.0,0.455451,0,0,0
2,69.322807,16.0,0.230472,0,0,0
3,10.987241,8.0,0.721550,0,0,0
4,87.889847,18.0,0.204569,0,0,0
5,14.463734,11.0,0.755301,0,0,0
6,45.000553,12.0,0.266072,0,0,0
7,43.226852,13.0,0.300045,0,0,0
8,43.183113,12.0,0.277244,0,0,0
9,48.276211,17.0,0.351412,0,0,0


Highlights extreme-risk areas. These features make it easy for dashboards to flag high-risk zones.


In [7]:
# Prepare features lists for modeling
numeric_features = [
    'lat','lon','elevation','precipitat',
    'cluster_mean_elev','cluster_mean_precip','cluster_mean_flood',
    'flood_risk_index_scaled','elev_precip_interaction',
    'low_elev_high_rain','very_low_elev','very_high_rain'
]
categorical_features = ['elevation_bin','precip_bin','cluster_id']

# Inspect feature lists
numeric_features, categorical_features


(['lat',
  'lon',
  'elevation',
  'precipitat',
  'cluster_mean_elev',
  'cluster_mean_precip',
  'cluster_mean_flood',
  'flood_risk_index_scaled',
  'elev_precip_interaction',
  'low_elev_high_rain',
  'very_low_elev',
  'very_high_rain'],
 ['elevation_bin', 'precip_bin', 'cluster_id'])

Separating numeric and categorical features ensures consistent preprocessing for models while keeping dashboards interpretable.


In [8]:
# Export refined dataset
processed_path = "C:/Users/SHANIA/Downloads/Metro-Manila-Flood-Insights/Data/AEGISDataset_processed_refined.csv"
df.to_csv(processed_path, index=False)
print(f"Refined dataset saved to '{processed_path}'")


Refined dataset saved to 'C:/Users/SHANIA/Downloads/Metro-Manila-Flood-Insights/Data/AEGISDataset_processed_refined.csv'


Refined dataset saved for modeling in Notebook 3, ensuring reproducibility and transparency.